In [1]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [2]:
from bert_serving.client import BertClient
bc = BertClient()

In [15]:
import re
import os
import json
import nltk
import gensim
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/rohan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rohan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

STOPWORDS = stopwords.words("english")

def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [6]:
import os
import pandas as pd
from nltk import sent_tokenize
count = 1

def extract_imperatives(reviews): #it takes a list of reviews and returns a list of imperatives
    imperatives = []
    
    for review in reviews:
        
        sentences = sent_tokenize(review)
    
        for sent in sentences: 
            result = nlp.annotate(sent,
                            properties={
                                'annotators': 'pos',
                                'outputFormat': 'json',
                                'timeout': 1000,
                            })
            try:
                if "VB" in result["sentences"][0]["tokens"][0]["pos"]:
                    imperatives.append(sent)
                
            except:
                pass
    return imperatives

In [32]:
def reminder(clean_reviews):
    words = []

    for sentence in clean_reviews:
        if len(sentence) == 0:
            continue
        temp = nltk.word_tokenize(sentence)
        words.extend(temp)
    if len(words) == 0:
        return None, None, None
    combined_sentence = [""]
    combined_sentence[0] = " ".join(words)
    words_vec = bc.encode(words)
    sent_vec = bc.encode(combined_sentence)
    similarities = cosine_similarity(sent_vec, words_vec)[0]

    sorted_list = np.argsort(similarities)[::-1]
    words_bert = []

    for k in sorted_list:
        if words[k] in words_bert or len(words_bert) >= 20:
            continue
        words_bert.append(words[k])
        
    words_tfidf = []

    vectorizer = TfidfVectorizer()

    X = vectorizer.fit_transform(clean_reviews)
    X = np.array(X.todense())
    X = np.mean(X, axis = 0)

    X = np.argsort(X)[::-1]

    temp = vectorizer.get_feature_names()

    for i in X:
        words_tfidf.append(temp[i])
    clean_reviews_tokens = []

    for i in clean_reviews:
        clean_reviews_tokens.append(i.split())

    dictionary = gensim.corpora.Dictionary(clean_reviews_tokens)
    bow_corpus = [dictionary.doc2bow(doc) for doc in clean_reviews_tokens]

    lda_model =  gensim.models.LdaMulticore(bow_corpus, num_topics = 10, id2word = dictionary, passes = 10, workers = 2)
    words_lda = []

    for idx, topic in lda_model.print_topics(-1):
        temp = topic.split('"')
        for i in range(len(temp)):
            if i%2 == 0 or temp[i] in words_lda:
                continue
            words_lda.append(temp[i])
    final_list = list(set(words_bert) & set(words_tfidf) & set(words_lda))

    return words_tfidf, words_lda, words_bert

In [33]:
import os
import pandas as pd
count = 1


for file_name in os.listdir('data/Recent'):
    print(count, 'of', len(os.listdir('data/Recent')))
    count += 1
    if file_name[:-4]+'_bert.csv' in os.listdir('bert_lda_tfidf'):
        continue
    df = pd.read_csv('data/Recent/' + file_name)
    reviews = df.iloc[:, 1].values
    imperatives = extract_imperatives(reviews)
    clean_imperatives = list(map(clean_text, imperatives))
    words_tfidf, words_lda, words_bert = reminder(clean_imperatives)
    if words_lda == None:
        continue
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    
    df1['bert'] = words_bert
    df2['tfidf'] = words_tfidf
    df3['lda'] = words_lda

    df1.to_csv('bert_lda_tfidf/'+file_name[:-4]+'_bert.csv', index=False)
    df2.to_csv('bert_lda_tfidf/'+file_name[:-4]+'_lda.csv', index=False)
    df3.to_csv('bert_lda_tfidf/'+file_name[:-4]+'_tfidf.csv', index=False)

1 of 339
2 of 339
3 of 339
4 of 339
5 of 339
6 of 339
7 of 339
8 of 339
9 of 339
10 of 339
11 of 339
12 of 339
13 of 339
14 of 339
15 of 339
16 of 339
17 of 339
18 of 339
19 of 339
20 of 339
21 of 339
22 of 339
23 of 339
24 of 339
25 of 339
26 of 339
27 of 339
28 of 339
29 of 339
30 of 339
31 of 339
32 of 339
33 of 339
34 of 339
35 of 339
36 of 339
37 of 339
38 of 339
39 of 339
40 of 339
41 of 339
42 of 339
43 of 339
44 of 339
45 of 339
46 of 339
47 of 339
48 of 339
49 of 339
50 of 339
51 of 339
52 of 339
53 of 339
54 of 339
55 of 339
56 of 339
57 of 339
58 of 339
59 of 339
60 of 339
61 of 339
62 of 339
63 of 339
64 of 339
65 of 339
66 of 339
67 of 339
68 of 339
69 of 339
70 of 339
71 of 339
72 of 339
73 of 339
74 of 339
75 of 339
76 of 339
77 of 339
78 of 339
79 of 339
80 of 339
81 of 339
82 of 339
83 of 339
84 of 339
85 of 339
86 of 339
87 of 339
88 of 339
89 of 339
90 of 339
91 of 339
92 of 339
93 of 339
94 of 339
95 of 339
96 of 339
97 of 339
98 of 339
99 of 339
100 of 339
101 of 3